In [1]:
import pandas as pd
import numpy as np 
import re 
import string
import warnings 
warnings.simplefilter(action='ignore',category=FutureWarning)
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer #to stem words
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax #to convert output into probability 

In [2]:
Palestine = pd.read_excel('E:\Work\Data Analysis Projects\Palestinian Project\Sentiment analysis files\\SentimentFile.xlsx',index_col=0)
Palestine.head()

,Tweet Id,DateTime,Place,Source,Username,Content,hashtags,Likes Number,Quote Number,Reply Number,Retweets Number,Month,Day,Hour,LANG
129368,1477081935184858880,2022-01-01,NaN,Android,SFOUK_AL_SHIKH,تحت الطاولة: بلغ حجم التبادل التجاري والاستثم...,"الجزائر, إسرائيل",75,1,9,9,1,1,0,Arabic
129369,1477077480573914880,2022-01-01,NaN,iPhone,IbtisamAzem,على أمل السنة الجاي أفضل من 2021 كل عام وأنتم...,"فلسطين, العراق, سورية, اليمن, مصر, تونس, السود...",10,0,5,0,1,1,0,Arabic
129370,1477076343296777984,2022-01-01,NaN,Android,AhmedBatneeji,كتيبة ثم سرايا ثم الجندي ثم شارع الوحدة واليوم...,No_Hash,32,0,0,0,1,1,0,Arabic
129371,1477075145340689920,2022-01-01,NaN,iPhone,MaherA7med,لحظة إيقاد شعلة ذكرى إنطلاقة الثورة الفلسطينية...,No_Hash,58,1,3,10,1,1,0,Arabic
129372,1477069104687333120,2022-01-01,NaN,Android,Rahmon83,وترجل رمح فلسطين العالي وشاعرها الكبير ابن جني...,No_Hash,50,0,5,7,1,1,0,Arabic


In [3]:
#Preparing DATA : 
VerEN = Palestine.loc[Palestine['LANG']=='English']
VerAR = Palestine.loc[Palestine['LANG']=='Arabic']
VersEN = VerEN[['Content','Tweet Id']]
VersEN

,Content,Tweet Id
71688,a photo from 2021 that will always ground me a...,1477076437651934976
71689,My 2022 wish: for world to embrace the incredi...,1477073720875311104
71690,I for one have never bought into the internati...,1477070281596408064
71691,I forgot to say that partner and kid made me a...,1477069177764545024
71692,Happy New Year friends. All we want for 2022 i...,1477067807644922112
...,...,...
7,There is a threat to Israel from Gaza. But it'...,1564755561652130048
8,"How could I forget the dreams, memories, and l...",1564755451090199040
9,"How could I forget the dreams, memories, and l...",1564755384673385984
10,Oh really? The same Christian cultural hegemon...,1564754597561908992


### English Sentiment Analysis  

In [4]:
import nltk

In [83]:
ps = PorterStemmer() #to remove extra letters at the end of the word ex : Lov-'ed'

In [84]:
VersEN.drop_duplicates(inplace=True,keep='first')

<ipython-input-84-a7e30e713f9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VersEN.drop_duplicates(inplace=True,keep='first')


In [85]:
#removing links
VersEN['Content'] = VersEN['Content'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

<ipython-input-85-63eb7d5ae894>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VersEN['Content'] = VersEN['Content'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


In [86]:
VersEN.reset_index(drop=True,inplace=True)

In [87]:

#Roberta Model
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [88]:
#Initializing Labels
labels = ['Negative','Neutral','Positive']

In [89]:
for i in range(len(VersEN)) : 
    s = re.sub('[^a-zA-Z]',' ',VersEN['Content'].iloc[i]) #to keep words only , ^ means Not 
    s = s.lower() # to lower case the words
    s = s.split() #ivides a String into an ordered list of substrings
    s = [word for word in s if word  not in stopwords.words('english')] #keep only words that are not in the stop words list 
    s = ' '.join(s) # joins words with white space in between
    s = ps.stem(s)
    encoded_tweet = tokenizer(s,return_tensors='pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    for j in range(len(scores)) : 
        if max(scores) == scores[j] : 
            VersEN.loc[i,'Sentiment'] = labels[j]

C:\Users\Madjid\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Madjid\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [90]:
VersEN

,Content,Tweet Id,Sentiment
0,a photo from 2021 that will always ground me a...,1477076437651934976,Positive
1,My 2022 wish: for world to embrace the incredi...,1477073720875311104,Positive
2,I for one have never bought into the internati...,1477070281596408064,Negative
3,I forgot to say that partner and kid made me a...,1477069177764545024,Neutral
4,Happy New Year friends. All we want for 2022 i...,1477067807644922112,Positive
...,...,...,...
71426,There is a threat to Israel from Gaza. But it'...,1564755561652130048,Neutral
71427,"How could I forget the dreams, memories, and l...",1564755451090199040,Positive
71428,"How could I forget the dreams, memories, and l...",1564755384673385984,Positive
71429,Oh really? The same Christian cultural hegemon...,1564754597561908992,Negative


In [92]:
VersEN.to_excel('E:\\Work\\Data Analysis Projects\\Palestinian Project\\VersEn.xlsx')

### Arabic Sentiment Analysis  

In [5]:
VersAR = VerAR[["Content",'Tweet Id']]

In [6]:
VersAR.dropna(axis = 0,inplace=True)
VersAR.isna().sum()

<ipython-input-6-eea8317ff42b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VersAR.dropna(axis = 0,inplace=True)


Content     0
Tweet Id    0
dtype: int64

In [7]:
VersAR.drop_duplicates(inplace=True,keep='first')

<ipython-input-7-502e1e158d43>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VersAR.drop_duplicates(inplace=True,keep='first')


In [8]:
VersAR.shape

(127614, 2)

In [9]:
#akhooli arabic model for Sentiment analysis
tokenizer1 = AutoTokenizer.from_pretrained('akhooli/xlm-r-large-arabic-sent')
model1 = AutoModelForSequenceClassification.from_pretrained("akhooli/xlm-r-large-arabic-sent")

Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# first we define a list of arabic and english punctiations that we want to get rid of in our text

punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words('arabic')

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    text = re.sub('[A-Za-z0-9]',' ',text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
  


In [11]:
VersAR.reset_index(inplace=True,drop=True)

In [12]:
labels1 = ['Neutral','Negative','Positive']

In [13]:
len(VersAR)

127614

In [15]:
from random import sample 
#making a sample with 99.99 % confidence and 0.5 % margin of error
VersAR = VersAR.sample(n=69231)
VersAR.reset_index(inplace=True,drop=True)
VersAR.head(10)

,Content,Tweet Id
0,تكفّلت الأطفال الأيتام من مجزرة دير ياسين، و ق...,1535198740876869888
1,اللهم كن لإخواننا في فلسطين وغزه عونا اللهم ال...,1556881938350318080
2,فيديو يوثق لحظة هروب جنود الاحتلال من حجارة ال...,1534050895209680896
3,بن غفير عضو الكنيست والمتشدد وفي ظل الاعتدائات...,1493543385822646016
4,#نفير_الأقصى سيعود الحق إلى أهله ما دام وراء ...,1530616162555420928
5,تعزّ علينا فلسطين تعز علينا أحزانها فسلامُ الل...,1515635054197850112
6,فلسطين الفرح ..,1512142051118362880
7,محمد شلح، أحد قادة حركة الجهاد الإسلامي في فلس...,1517105578434253056
8,"ماذا يملك العدو مقابل عزّمنا؟ ""عزّمنا القسّام...",1538479157172219904
9,يافا عروس بحر فلسطين 🇵🇸💙 :// . / 5 1 3,1527019843416953088


In [1]:
#running the Sentiments Model
for i in range(len(VersAR)) :
    print(i)
    s1 = preprocess(VersAR.loc[i,'Content'])
    encoded_tweet1 = tokenizer1(s1,return_tensors='pt')
    output1 = model1(**encoded_tweet1)
    scores1 = output1[0][0].detach().numpy()
    scores1 = softmax(scores1)
    for j in range(len(scores1)) : 
        if max(scores1) == scores1[j] : 
            VersAR.loc[i,'Sentiment'] = labels1[j]


NameError: name 'VersAR' is not defined

In [17]:
VersAR.to_excel('E:\\Work\\Data Analysis Projects\\Palestinian Project\\VersAR.xlsx')

In [18]:
VersAR.loc[VersAR["Sentiment"].notnull()]

,Content,Tweet Id,Sentiment
0,تكفّلت الأطفال الأيتام من مجزرة دير ياسين، و ق...,1535198740876869888,Positive
1,اللهم كن لإخواننا في فلسطين وغزه عونا اللهم ال...,1556881938350318080,Positive
2,فيديو يوثق لحظة هروب جنود الاحتلال من حجارة ال...,1534050895209680896,Positive
3,بن غفير عضو الكنيست والمتشدد وفي ظل الاعتدائات...,1493543385822646016,Neutral
4,#نفير_الأقصى سيعود الحق إلى أهله ما دام وراء ...,1530616162555420928,Positive
...,...,...,...
69226,كمية اللجان الالكترونيه الصفحات الجديده الي دخ...,1556529326820146944,Negative
69227,مع #غزة نُصرةً ل #فلسطين,1555557912700935936,Positive
69228,"الجيش الإسرائيلي يقرر خطة "" إخلاء سكان غلاف غز...",1555584868582625024,Positive
69229,تحييد حماس عن العدوان الإسرائيلي على غزة المسم...,1555661914788724992,Positive
